In [185]:
import pandas as pd
from openpyxl import Workbook
from openpyxl import load_workbook
from google.cloud import translate
import google.auth
import itertools

### Load Excel Workbook

In [184]:
fname = 'test'
wb = load_workbook(fname+'.xlsx')
#print(wb.sheetnames)
ws = wb['Spanish']

### Library to store translated phrases ... to avoid uneccesary API calls

In [176]:
translated = {}

### Setup Google Translate API Client

In [186]:
# this notebook uses googles default credentials 
credentials, project = google.auth.default()

# api client to manage translate calls
translate_client = translate.Client(credentials=credentials)

In [187]:
# funtion to determine if phrase contains any characters
def anychar(value):
    if isinstance(value, basestring):
        clean = ''.join(e for e in value.encode('utf8') if e.isalpha())
        if len(clean) > 0:
            return True
        else:
            return False
    else:
        return False

In [188]:
# function to call google api tranlastion if phrase contains letters AND hasn't already been translated
def translate(phrase, from_lang='es', to_lang='en'):
    print "-"*40
    print "TRANSLATE: ", phrase
    print
    
    if anychar(phrase)==False:
        print "No Characters to Translate"
        print phrase
        print
        return phrase
        
    elif phrase in translated:
        print "Already Translated"
        print translated[phrase]['translatedText']
        print
        return translated[phrase]['translatedText']
        
    else:
        t = translate_client.translate(phrase, target_language=to_lang, format_='text')
        translated[phrase] = t
        print "New Translation"
        print t['translatedText']
        print
        return t['translatedText']

### Iterate Worksheet and Translate Cells

In [191]:
for col in ws.iter_cols():
    for cell in col:
        
        if cell.col_idx == 2:
            #cell.value = 'new - ' + cell.value.encode('utf-8')
            cell.value = translate(cell.value)
        
        
        #print(cell.value, cell.anchor, cell.row, cell.col_idx)

----------------------------------------
TRANSLATE:  Spanish

Already Translated
Spanish

----------------------------------------
TRANSLATE:  Buenos días.

Already Translated
Good Morning.

----------------------------------------
TRANSLATE:  Buenas tardes.

Already Translated
Good afternoon.

----------------------------------------
TRANSLATE:  Buenas noches.

Already Translated
Goodnight.

----------------------------------------
TRANSLATE:  Hola, me llamo Juan.

Already Translated
Hello, my name is Juan.

----------------------------------------
TRANSLATE:  ¿Cómo se llama usted?

Already Translated
What is your name?

----------------------------------------
TRANSLATE:  ¿Cómo está usted?

Already Translated
How are you?

----------------------------------------
TRANSLATE:  Estoy bien.

Already Translated
I'm fine.

----------------------------------------
TRANSLATE:  Mucho gusto.

Already Translated
Pleasure.

----------------------------------------
TRANSLATE:  Adiós.

Already Tra

In [192]:
wb.save(fname+'_translated.xlsx')